In [1]:
import geomesa_pyspark 
conf = geomesa_pyspark.configure(jars=['/usr/lib/spark/jars/geomesa-hbase-spark-runtime_2.11-2.1.0-m.2.jar'],
                                 packages=['geomesa_pyspark','pytz'], 
                                 spark_home='/usr/lib/spark/') 
#setAppName('MyTestApp')
conf.get('spark.master') # u'yarn'
from pyspark.sql import SparkSession
spark = ( SparkSession.builder.config(conf=conf).getOrCreate() )


Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
78,application_1599127947832_0079,pyspark3,idle,Link,Link,✔


SparkSession available as 'spark'.


In [5]:
params = {    "hbase.zookeepers": "hbase.optix-ons-local:2181",    "hbase.catalog": "ons-historical" } 
feature = "ee" 
ee = ( spark    .read    .format("geomesa")    .options(**params)    .option("geomesa.feature", feature)    .load() )
ee.createOrReplaceTempView("ee")


In [6]:
df1=spark.sql(""" select dtg, vessel_name, message_type from ee where 
           st_intersects(position,st_makeBox2d(st_point(-46.481607,-24.1178),st_point(-46.1763,-23.8787))) AND
           dtg > cast('2020-09-01 00:00' as timestamp) AND message_type = 12 """).coalesce(1).write.option("header", "true").csv("s3://optix.ons.jupyter/jupyter/dwater/folder/dados")
